# Dense Matrix-Vector Product, 2D Decomposition

This assignment will be graded on four nodes with 28 cores, but for development you can run anywhere.

In this assignment you will complete the dense matrix vector product code where the matrix is decomposed by a 2D decomposition.

In [ ]:
module unload cse6230
module load cse6230/gcc-omp-gpu

In [ ]:
make clean
make dense_matrix_vector_product